In [1]:
import numpy as np

class LegendreDecomposition:
    """
    Legendre Decomposition for Tensors using the Natural Gradient method.

    This class implements the algorithm described in the paper "Legendre
    Decomposition for Tensors" (arXiv:1802.04502v2).
    """

    def __init__(self, n_iter: int = 20, tol: float = 1e-7):
        """
        Args:
            n_iter (int): Maximum number of iterations.
            tol (float): Tolerance for the norm of the gradient to determine convergence.
        """
        self.n_iter = n_iter
        self.tol = tol
        self.P_norm_ = None
        self.Q_ = None
        self.theta_ = None
        self.basis_ = None
        self.eta_hat_ = None
        self.zeta_matrix_ = None
        self.omega_ = None
        self.shape_ = None
        self.rmse_history_ = []

    def _precompute_zeta_matrix(self):
        """
        Precomputes the matrix representing the partial order relationship.
        
        zeta_matrix[i, j] = 1 if basis[i] <= omega[j], and 0 otherwise.
        This matrix is central to both decoding (theta -> Q) and encoding (Q -> eta).
        """
        num_basis = len(self.basis_)
        num_omega = len(self.omega_)
        
        # Expand basis and omega for vectorized comparison
        basis_expanded = np.array(self.basis_)[:, np.newaxis, :] # Shape: (num_basis, 1, n_dims)
        omega_expanded = np.array(self.omega_)[np.newaxis, :, :] # Shape: (1, num_omega, n_dims)

        # ζ(u,v) = 1 if u <= v. u is from basis, v is from omega.
        # This checks if for every dimension, basis_coord <= omega_coord.
        # The result is a boolean matrix of shape (num_basis, num_omega).
        le_matrix = np.all(basis_expanded <= omega_expanded, axis=2)
        
        self.zeta_matrix_ = le_matrix.astype(int)

    def fit(self, P: np.ndarray, basis: list[tuple]):
        """
        Decomposes the input tensor P using the given basis.

        Args:
            P (np.ndarray): The non-negative input tensor.
            basis (list[tuple]): A list of index tuples representing the basis B.
                                 e.g., [(0, 1), (1, 0)] for a 2D tensor.
        """
        # --- 0. Initialization and Pre-computation ---
        self.shape_ = P.shape
        self.basis_ = basis
        
        # The sample space Ω is the set of all indices
        self.omega_ = [idx for idx, _ in np.ndenumerate(P)]
        
        # Normalize the input tensor P to make it a probability distribution
        p_sum = np.sum(P)
        if p_sum == 0:
            raise ValueError("Input tensor P cannot be all zeros.")
        self.P_norm_ = P / p_sum
        
        p_flat = self.P_norm_.flatten()
        
        # This is the most important pre-computation step
        self._precompute_zeta_matrix()
        
        # Initialize parameters θ (theta)
        self.theta_ = np.zeros(len(self.basis_))
        
        # Compute target expectation η̂ (eta-hat) from P. This is done only once.
        # η̂_v = Σ_{u∈Ω, v≤u} p_u  (Eq. 3 with P)
        self.eta_hat_ = self.zeta_matrix_ @ p_flat
        
        self.rmse_history_ = []

        print(f"Starting optimization for tensor of shape {self.shape_} with {len(self.basis_)} basis elements.")

        # --- Main Optimization Loop (Natural Gradient) ---
        for i in range(self.n_iter):
            # --- 1. Decode: Compute Q from current θ (theta) ---
            # log q_v = Σ_{u∈B, u≤v} θ_u - ψ(θ) (Eq. 2)
            # The sum part is a matrix-vector product: ζ^T * θ
            log_q_unnormalized = self.zeta_matrix_.T @ self.theta_
            
            # Use LogSumExp trick for numerical stability (part of softmax)
            log_q_unnormalized -= np.max(log_q_unnormalized)
            q_temp = np.exp(log_q_unnormalized)
            q_flat = q_temp / np.sum(q_temp)
            
            # --- 2. Encode: Compute η (eta) from Q ---
            # η_v = Σ_{u∈Ω, v≤u} q_u (Eq. 3)
            # This is a matrix-vector product: ζ * q
            eta = self.zeta_matrix_ @ q_flat
            
            # --- 3. Compute Gradient Δη ---
            delta_eta = eta - self.eta_hat_
            
            # --- 4. Check for Convergence ---
            grad_norm = np.linalg.norm(delta_eta)
            self.Q_ = q_flat.reshape(self.shape_) * p_sum # Rescale to original magnitude
            rmse = np.sqrt(np.mean((P - self.Q_)**2))
            self.rmse_history_.append(rmse)
            
            print(f"Iter {i+1:2d}: Grad Norm = {grad_norm:.4e}, RMSE = {rmse:.4f}")
            if grad_norm < self.tol:
                print("Convergence reached.")
                break

            # --- 5. Compute Fisher Information Matrix G ---
            # G_uv = Cov(ζ_u, ζ_v) = Σ_w q_w ζ(u,w)ζ(v,w) - η_uη_v (Eq. 5)
            # Vectorized form: G = ζ * diag(q) * ζ^T - η * η^T
            G = (self.zeta_matrix_ * q_flat) @ self.zeta_matrix_.T - np.outer(eta, eta)

            # --- 6. Update θ (theta) ---
            # θ_next = θ - G⁻¹ * Δη
            # Use np.linalg.solve for better stability and performance
            try:
                # Add a small regularization term (ridge) for stability
                reg = 1e-8 * np.eye(G.shape[0])
                update_step = np.linalg.solve(G + reg, delta_eta)
                self.theta_ -= update_step
            except np.linalg.LinAlgError:
                print("Warning: Fisher matrix is singular. Using pseudo-inverse.")
                update_step = np.linalg.pinv(G) @ delta_eta
                self.theta_ -= update_step
                
        # Final reconstructed tensor
        self.Q_ = q_flat.reshape(self.shape_) * p_sum
        
        return self

In [4]:
# 1. Create a synthetic 2D tensor (a 4x5 matrix)
P = np.array([
    [10, 12, 5, 2, 1],
    [15, 20, 8, 3, 2],
    [8,  10, 15, 9, 4],
    [3,  4,  11, 8, 5]
])

# 2. Define a basis B. 
# Let's choose a basis that only includes the first row and first column.
# This corresponds to a low-rank approximation in the log-domain.
# Note: Using 0-based indexing.
basis_elements = [(i, 0) for i in range(P.shape[0])] + \
                    [(0, j) for j in range(1, P.shape[1])] # Avoid adding (0,0) twice

# 3. Create a model and fit it to the data
ld_model = LegendreDecomposition(n_iter=100, tol=1e-10)
ld_model.fit(P, basis=basis_elements)

# 4. Get the reconstructed tensor
Q = ld_model.Q_

final_rmse = np.sqrt(np.mean((P - Q)**2))

print("\n--- Results ---")
print("Original Tensor P:\n", np.round(P, 2))
print("\nReconstructed Tensor Q:\n", np.round(Q, 2))
print(f"\nFinal RMSE: {final_rmse:.4f}")

Starting optimization for tensor of shape (4, 5) with 8 basis elements.
Iter  1: Grad Norm = 2.6654e-01, RMSE = 4.9787
Iter  2: Grad Norm = 3.5190e-02, RMSE = 3.4138
Iter  3: Grad Norm = 3.1348e-03, RMSE = 3.3733
Iter  4: Grad Norm = 4.1308e-05, RMSE = 3.3720
Iter  5: Grad Norm = 6.8692e-09, RMSE = 3.3719
Iter  6: Grad Norm = 8.4791e-16, RMSE = 3.3719
Convergence reached.

--- Results ---
Original Tensor P:
 [[10 12  5  2  1]
 [15 20  8  3  2]
 [ 8 10 15  9  4]
 [ 3  4 11  8  5]]

Reconstructed Tensor Q:
 [[ 6.97  8.9   7.55  4.26  2.32]
 [11.15 14.25 12.08  6.81  3.72]
 [10.68 13.65 11.57  6.53  3.56]
 [ 7.2   9.2   7.8   4.4   2.4 ]]

Final RMSE: 3.3719


In [5]:
# 例：値が大きい上位10個のインデックスを基底に追加する

# テンソルの各要素とそのインデックスを平坦化
flat_P = P.flatten()
indices = [idx for idx, _ in np.ndenumerate(P)]

# 値の大きい順にソート
sorted_indices = sorted(zip(flat_P, indices), key=lambda x: x[0], reverse=True)

# 上位10個のインデックスを取得
top_10_indices = [idx for val, idx in sorted_indices[:10]]

# これを基底に追加する
new_basis = list(set(basis_elements + top_10_indices))

# 3. Create a model and fit it to the data
ld_model = LegendreDecomposition(n_iter=100, tol=1e-10)
ld_model.fit(P, basis=new_basis)

# 4. Get the reconstructed tensor
Q = ld_model.Q_

final_rmse = np.sqrt(np.mean((P - Q)**2))

print("\n--- Results ---")
print("Original Tensor P:\n", np.round(P, 2))
print("\nReconstructed Tensor Q:\n", np.round(Q, 2))
print(f"\nFinal RMSE: {final_rmse:.4f}")

Starting optimization for tensor of shape (4, 5) with 14 basis elements.
Iter  1: Grad Norm = 2.7654e-01, RMSE = 4.9787
Iter  2: Grad Norm = 1.1325e-01, RMSE = 2.6085
Iter  3: Grad Norm = 1.0684e-02, RMSE = 0.5113
Iter  4: Grad Norm = 1.9029e-04, RMSE = 0.4285
Iter  5: Grad Norm = 1.0094e-07, RMSE = 0.4285
Iter  6: Grad Norm = 6.2807e-14, RMSE = 0.4285
Convergence reached.

--- Results ---
Original Tensor P:
 [[10 12  5  2  1]
 [15 20  8  3  2]
 [ 8 10 15  9  4]
 [ 3  4 11  8  5]]

Reconstructed Tensor Q:
 [[10.   12.    4.95  1.97  1.08]
 [15.   20.    8.05  3.2   1.75]
 [ 7.92 10.08 14.    9.06  4.94]
 [ 3.08  3.92 12.    7.76  4.24]]

Final RMSE: 0.4285
